In [ ]:
import os
import json
import numpy as np
from tifffile import imread
import cv2
import skimage.io as sio

import albumentations as A
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision.models.detection import MaskRCNN, FasterRCNN_ResNet50_FPN_Weights, MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models import ResNet50_Weights
from torchvision.ops import box_convert
import torchvision.transforms as T
import torch.nn.functional as F

import torch
from torch.optim import SGD, lr_scheduler

import pathlib
import json

from tqdm import tqdm
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask
from pycocotools.cocoeval import COCOeval

from PIL import Image

In [25]:
# !gdown https://drive.google.com/file/d/1B0qWNzQZQmfQP7x7o4FDdgb9GvPDoFzI/view --fuzzy
# !mkdir ../dataset
# !tar -xzf hw3-data-release.tar.gz
# !mv test_release/ ../dataset
# !mv train/ ../dataset/
# !mv test_image_name_to_ids.json ../dataset/

In [ ]:
import numpy as np
import skimage.io as sio
from pycocotools import mask as mask_utils


def decode_maskobj(mask_obj):
    return mask_utils.decode(mask_obj)


def encode_mask(binary_mask):
    arr = np.asfortranarray(binary_mask).astype(np.uint8)
    rle = mask_utils.encode(arr)
    rle['counts'] = rle['counts'].decode('utf-8')
    return rle


def read_maskfile(filepath):
    mask_array = sio.imread(filepath)
    return mask_array

In [26]:
class MedicalDataset(Dataset):
    def __init__(self, root_dir, transform=None, data_type='Train'):
        self.root = root_dir
        self.transform = transform
        self.data_type = data_type
        if self.data_type not in ['Train', 'Valid', 'Test']:
            raise ValueError('Data type should be in [Train, Valid, Test]')
        self.samples = self._load_samples()

        self.train_coco_path = os.path.join(pathlib.Path(root_dir).parent, 'train_coco.json')
        self.val_coco_path = os.path.join(pathlib.Path(root_dir).parent, 'val_coco.json')
        if not os.path.exists(self.train_coco_path) or not os.path.exists(self.val_coco_path):
            # self.generate_coco(self.train_coco_path)
            self.generate_coco_split(self.train_coco_path, self.val_coco_path, split_ratio=0.2)
        self.train_coco = COCO(self.train_coco_path)
        self.val_coco = COCO(self.val_coco_path)
        self.num_classes = len(self.train_coco.loadCats(self.train_coco.getCatIds()))

    def _load_samples(self):
        samples = []
        for img_dir in os.listdir(self.root):
            tmp_dir = os.path.join(self.root, img_dir)

            if self.data_type == 'Train' or self.data_type == 'Valid':
                img_path = os.path.join(tmp_dir, 'image.tif')

                mask_paths = [
                    entry.name for entry in pathlib.Path(tmp_dir).iterdir()
                    if entry.name.startswith("class") and entry.is_file()
                ]

                samples.append({'image': img_path, 'masks': mask_paths})
            elif self.data_type == 'Test':
                test_img_json_path = os.path.join(pathlib.Path(self.root).parent, 'test_image_name_to_ids.json')
                with open(test_img_json_path, 'r') as f:
                    samples = json.load(f)

            else:
                raise ValueError('Wrong data type')

                # for idx in range(len(samples)):
                #     samples[idx]['file_name'] = os.path.join(self.root, samples[idx]['file_name'])
        return samples

    def mask_to_polygons(self, mask, epsilon=1.0):
        contours,_ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        polygons = []
        for contour in contours:
            if len(contour) > 2:
                poly = contour.reshape(-1).tolist()
                if len(poly) > 4: #Ensures valid polygon
                    polygons.append(poly)
        return polygons

    def generate_coco(self, output_dir, train=True):
        annotations = []
        images = []
        categories = []
        all_labels = []
        ann_id = 0

        for img_id, sample in enumerate(self.samples):
            print(f'({img_id}/{len(self.samples)})')
            img_path, mask_paths = sample['image'], sample['masks']
            img = cv2.imread(img_path)
            masks = [cv2.imread(os.path.join(pathlib.Path(img_path).parent, mask_path), cv2.IMREAD_UNCHANGED) for mask_path in mask_paths]

            images.append({
                "id": img_id,
                "file_name": img_path,
                "height": img.shape[0],
                "width": img.shape[1]
            })

            for mask in masks:
                unique_values = np.unique(mask)
                all_labels.append(unique_values)
                for value in unique_values:
                    if value == 0:  # Ignore background
                        continue

                    object_mask = (mask == value).astype(np.uint8) * 255
                    polygons = self.mask_to_polygons(object_mask)

                    for poly in polygons:
                        ann_id += 1
                        annotations.append({
                            "id": ann_id,
                            "image_id": img_id,
                            "category_id": 1,  # Only one category: Nuclei
                            "segmentation": [poly],
                            "area": cv2.contourArea(np.array(poly).reshape(-1, 2)),
                            "bbox": list(cv2.boundingRect(np.array(poly).reshape(-1, 2))),
                            "iscrowd": 0
                        })

        all_labels = np.unique(np.concatenate(all_labels).tolist())

        for idx, label in enumerate(all_labels):
            categories.append({"id": idx+1, "name": int(label)})

        coco_input = {
            "images": images,
            "annotations": annotations,
            "categories": categories
        }

        print(f'Saving train coco json')

        with open(output_dir, 'w') as f:
            json.dump(coco_input, f)

    def generate_coco_split(self, train_coco_path, val_coco_path, split_ratio=0.2):
        train_data = {"images": [], "annotations": [], "categories": []}
        val_data = {"images": [], "annotations": [], "categories": []}
        all_labels = []
        ann_id = 0

        # 隨機分離樣本索引
        indices = list(range(len(self.samples)))
        import random
        seed = 123
        random.Random(seed).shuffle(indices)
        split_point = int(len(indices) * split_ratio)
        train_indices = indices[:split_point]
        val_indices = indices[split_point:]

        # 類別統一管理 (避免訓練/驗證類別不一致)
        global_categories = {}

        for dataset_type, indices in [("train", train_indices), ("val", val_indices)]:
            target_data = train_data if dataset_type == "train" else val_data
           
            for idx in indices:
                sample = self.samples[idx]
                img_path, mask_paths = sample['image'], sample['masks']
                img = cv2.imread(img_path)
                masks = [cv2.imread(os.path.join(pathlib.Path(img_path).parent, mask_path), cv2.IMREAD_UNCHANGED) for mask_path in mask_paths]

                image_entry = {
                    "id": idx,
                    "file_name": img_path,
                    "height": img.shape[0],
                    "width": img.shape[1]
                }
                target_data["images"].append(image_entry)

                for mask in masks:
                    unique_values = np.unique(mask)
                    all_labels.append(unique_values)
                    for value in unique_values:
                        if value == 0:  # Ignore background
                            continue

                        object_mask = (mask == value).astype(np.uint8) * 255
                        polygons = self.mask_to_polygons(object_mask)

                        for poly in polygons:
                            ann_id += 1
                            target_data["annotations"].append({
                                "id": ann_id,
                                "image_id": idx,
                                "category_id": int(value),  # Only one category: Nuclei
                                "segmentation": [poly],
                                "area": cv2.contourArea(np.array(poly).reshape(-1, 2)),
                                "bbox": list(cv2.boundingRect(np.array(poly).reshape(-1, 2))),
                                "iscrowd": 0
                            })

        all_labels = np.unique(np.concatenate(all_labels).tolist())

        categories = []
        for idx, label in enumerate(all_labels):
            categories.append({"id": idx+1, "name": int(label)})
        train_data["categories"] = categories
        val_data["categories"] = categories


        # coco_input = {
        #     "images": images,
        #     "annotations": annotations,
        #     "categories": categories
        # }

        print(f'Saving  coco json')

        with open(train_coco_path, 'w') as f:
            json.dump(train_data, f)
        with open(val_coco_path, 'w') as f:
            json.dump(val_data, f)


    def poly2mask(self, segmentation, img_size):
        """
        多邊形標註轉二值掩碼
        :param segmentation: COCO格式的多邊形坐標列表 [[x1,y1,x2,y2,...]]
        :param img_size: 目標圖像尺寸 (height, width)
        """
        # 自動檢測標註類型
        if isinstance(segmentation, dict):
            # 處理RLE格式
            return coco_mask.decode(segmentation)
        else:
            # 處理多邊形格式
            rle = coco_mask.frPyObjects(segmentation, img_size[0], img_size[1])
            return coco_mask.decode(rle)

    def __getitem__(self, index):
        if self.data_type == 'Train' or self.data_type == 'Valid':
            coco_file = self.train_coco if self.data_type == 'Train' else self.val_coco
            img_id = coco_file.dataset['images'][index]['id']
            img_ids = coco_file.getImgIds(imgIds=img_id)
            img_info = coco_file.loadImgs(img_ids)
            # image = cv2.imread(img_info[0]['file_name']) / 255.0
            print(img_info)
            image = Image.open(img_info[0]['file_name']).convert("RGB")
            image = self.transform(image) if self.transform is not None else image
            img_size = [img_info[0]['height'], img_info[0]['width']]


            boxes = []
            masks = []
            labels = []
            ann_ids = coco_file.getAnnIds(imgIds=img_ids)
            annotations = coco_file.loadAnns(ann_ids)
            for ann in annotations:
                boxes.append(ann['bbox'])
                tmp_mask = self.poly2mask(ann['segmentation'], img_size).squeeze()
                # mask_ = F.interpolate(
                #     tmp_mask,
                #     size=(224, 224),
                #     mode='nearest-exact'  # PyTorch 1.10+ 專用選項
                # )
                mask_ = cv2.resize(
                    tmp_mask,
                    (224, 224),
                    interpolation=cv2.INTER_NEAREST_EXACT  # 精確最近鄰算法
                )
                masks.append(mask_)
                labels.append(ann["category_id"])

            boxes = self.resize_box(boxes, img_size, target_size=[224,224])
            boxes = box_convert(torch.tensor(boxes, dtype=torch.float32), in_fmt='xywh', out_fmt='xyxy')
            masks = torch.as_tensor(np.array(masks), dtype=torch.bool)

            target = {'boxes': torch.as_tensor(boxes, dtype=torch.float32),
                      'masks': masks,
                      'labels': torch.as_tensor(np.array(labels), dtype=torch.int64)}

            return image, target
        else:
            raise ValueError('This is test, not yet implement')

    def resize_box(self, boxes, orig_size, target_size):
        # Eat xywh
        scale_w = target_size[1] / orig_size[1]
        scale_h = target_size[0] / orig_size[0]

        for box in boxes:
            box[0] *= scale_w  # x
            box[1] *= scale_h  # y
            box[2] *= scale_w  # w
            box[3] *= scale_h  # h

        return boxes

    def __len__(self):
        coco_file = self.train_coco if self.data_type == 'Train' else self.val_coco
        return len(coco_file.dataset['images'])

In [27]:
project_root = '..'
train_dir = os.path.join(project_root, 'dataset/train')
test_dir = os.path.join(project_root, 'dataset/test_release')

In [28]:
# train_coco_path = f'/home/bhg/visual_dl/lab3/dataset'
# val_coco_path = f'/home/bhg/visual_dl/lab3/dataset'
# train_set = MedicalDataset(root_dir=train_dir, data_type='Train')
# val_transform=T.Compose([
#     T.ToTensor(),
#     T.Resize(size=[224,224], antialias=True),
#     # T.CenterCrop(size=224),
#     # T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])
# val_set = MedicalDataset(root_dir=train_dir, data_type='Valid', transform=val_transform)

# print(val_set[1])

In [29]:
# train_transform=T.Compose([
#     T.ToTensor(),
#     T.Resize(size=[224,224], antialias=True),
#     # T.CenterCrop(size=224),
#     # T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])
# train_set = MedicalDataset(root_dir=train_dir, transform=train_transform)
# img, target = train_set[1]
# print(f"box: {target['boxes'].shape}")
# print(f"mask: {target['masks'].shape}")
# print(f"label: {target['labels'].shape}")
# print(target['boxes'][0])

# print(img, img.shape)

In [30]:
# train_transform = A.Compose([
#     A.HorizontalFlip(p=0.5),
#     A.VerticalFlip(p=0.3),
#     A.Rotate(limit=15, p=0.4),
#     A.CLAHE(p=0.5),
#     A.GridDistortion(p=0.2),
#     A.RandomBrightnessContrast(p=0.3)
# ], additional_targets={'mask': 'mask'})
train_transform=T.Compose([
    T.ToTensor(),
    T.Resize(size=[224, 224], antialias=True),
    # T.CenterCrop(size=224),
    # T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = train_transform

train_set = MedicalDataset(root_dir=train_dir, transform=train_transform, data_type='Train')
print(train_set[0])
val_set = MedicalDataset(root_dir=train_dir, transform=val_transform, data_type='Valid')
print(val_set[0])
# def custom_collate(batch):
#     images = []
#     targets = []

#     for img, target in batch:
#         images.append(img)
#         targets.append({
#             'boxes': target['boxes'],
#             'labels': target['labels'],
#             'masks': target['masks']
#         })

#     images = torch.stack(images, dim=0)
#     return images, targets


loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!
[{'id': 98, 'file_name': '../dataset/train/e3691953-56da-4756-b116-ef84a406672d/image.tif', 'height': 205, 'width': 382}]
(tensor([[[0.8450, 0.8818, 0.8521,  ..., 0.8589, 0.8288, 0.7991],
         [0.8632, 0.8987, 0.8641,  ..., 0.8484, 0.8108, 0.7860],
         [0.8785, 0.9104, 0.8766,  ..., 0.8403, 0.8018, 0.7843],
         ...,
         [0.8566, 0.8896, 0.9479,  ..., 0.7513, 0.7733, 0.7861],
         [0.8948, 0.8936, 0.9391,  ..., 0.7621, 0.7779, 0.7938],
         [0.9009, 0.8965, 0.9355,  ..., 0.7758, 0.7839, 0.8044]],

        [[0.8208, 0.8520, 0.8136,  ..., 0.7916, 0.7554, 0.7143],
         [0.8498, 0.8704, 0.8272,  ..., 0.7805, 0.7392, 0.7065],
         [0.8705, 0.8802, 0.8384,  ..., 0.7703, 0.7297, 0.7042],
         ...,
         [0.7593, 0.8279, 0.9112,  ..., 0.6649, 0.6870, 0.7020],
         [0.8131, 0.8276, 0.901

In [31]:
# MAX_BOXES_PER_BATCH = 5  # 依GPU記憶體調整

# def custom_collate(batch):
#     images = []
#     targets = []
#     residual_boxes = []  # 暫存未處理的boxes

#     for img, target in batch:
#         # 合併殘留框與新框
#         all_boxes = residual_boxes + target['boxes']

#         # 切割可用區塊
#         use_boxes = all_boxes[:MAX_BOXES_PER_BATCH]
#         residual_boxes = all_boxes[MAX_BOXES_PER_BATCH:]

#         # 僅在有可用框時加入當前批次
#         if len(use_boxes) > 0:
#             images.append(img)
#             targets.append({
#                 'boxes': use_boxes,
#                 'labels': target['labels'][:len(use_boxes)],
#                 'masks': target['masks'][:len(use_boxes)]
#             })

#     # 遞歸處理殘留框
#     if residual_boxes:
#         dummy_img = torch.zeros_like(img)  # 填充虛擬數據
#         return custom_collate([(dummy_img, {'boxes': residual_boxes})] + batch)

#     images = torch.stack(images)
#     return images, targets
def custom_collate(batch):
    images = []
    targets = []

    for img, target in batch:
        images.append(img)
        targets.append({
            'boxes': target['boxes'],
            'labels': target['labels'],
            'masks': target['masks']
        })

    images = torch.stack(images, dim=0)
    return images, targets


BATCH_SIZE = 1
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=False, collate_fn=custom_collate)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=False, collate_fn=custom_collate)

In [32]:
def evaluate_model(model, val_loader, device):
    # 初始化 COCO 格式儲存器
    coco_gt = COCO()  # 需提前加載驗證集註解文件
    coco_results = []
    
    model.eval()
    with torch.no_grad():
        for images, targets in val_loader:
            images = list(img.to(device) for img in images)
            outputs = model(images)
            
            # 轉換預測結果到 COCO 格式
            for i, output in enumerate(outputs):
                image_id = targets[i]["image_id"].item()
                
                # 處理每個實例預測
                for j in range(len(output["boxes"])):
                    box = output["boxes"][j].cpu().numpy()
                    score = output["scores"][j].item()
                    label = output["labels"][j].item()
                    mask = output["masks"][j][0].cpu().numpy()  # (H,W)
                    
                    # 生成 RLE 編碼 (COCO 要求格式)
                    rle = encode_mask(mask > 0.5)  # 閾值處理
                    
                    coco_results.append({
                        "image_id": image_id,
                        "category_id": label,
                        "segmentation": rle,
                        "bbox": [box[0], box[1], box[2]-box[0], box[3]-box[1]],  # xywh
                        "score": score
                    })
    
    # 評估計算
    coco_dt = coco_gt.loadRes(coco_results)
    coco_eval = COCOeval(coco_gt, coco_dt, 'segm')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    
    return coco_eval.stats  # 返回 AP 系列指標

In [33]:
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_V2_Weights
model = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT)
model.to('cuda')
params = [p for p in model.parameters() if p.requires_grad]
optimizer = SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
num_epochs = 1

from tqdm import tqdm


for epoch in tqdm(range(num_epochs), desc="Epochs"):
    print(f'a')
    model.train()
    bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    for images, targets in bar:
        # print('b')
        images = [image.to('cuda') for image in images]
        targets = [{k: v.to('cuda') for k, v in t.items()} for t in targets]
        # import torch.nn.functional as F
        # images = F.interpolate(
        #     images,
        #     size=224,
        #     mode='bilinear',
        #     align_corners=False
        # )

        # targets['masks'] = F.interpolate(
        #     targets['masks'],
        #     size=224,
        #     mode='nearest'
        # )

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        bar.set_postfix(loss=losses.detach().cpu().item())
        bar.update()

    evaluate_model(model, val_loader, 'cuda')

    lr_scheduler.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {losses.item()}")

torch.save(model.state_dict(), 'model.pth')

Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

a


[{'id': 126, 'file_name': '../dataset/train/ed834742-aab1-49df-8577-94568f3a581b/image.tif', 'height': 706, 'width': 859}]
[{'id': 50, 'file_name': '../dataset/train/61d7db4f-6dac-4773-9851-109aa165e72c/image.tif', 'height': 351, 'width': 694}]
[{'id': 6, 'file_name': '../dataset/train/10352a94-c1bd-419d-a2cf-7a4c486b61f8/image.tif', 'height': 902, 'width': 1220}]
[{'id': 30, 'file_name': '../dataset/train/9e0fbc88-ab03-4e48-83d8-26713290c1b6/image.tif', 'height': 1956, 'width': 1398}]
[{'id': 115, 'file_name': '../dataset/train/3ef561e2-3851-40f3-beff-1e5a4dfd81c2/image.tif', 'height': 1422, 'width': 2162}]


/pytorch/aten/src/ATen/native/cuda/Loss.cu:245: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [15,0,0] Assertion `t >= 0 && t < n_classes` failed.
Epochs:   0%|          | 0/1 [00:02<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN_ResNet50_FPN_V2_Weights
model = maskrcnn_resnet50_fpn_v2(weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT)
model.to('cuda')
model.eval()

from tqdm import tqdm
from torchvision.ops import box_convert

bar = tqdm(train_loader, desc="Inference", total=len(train_loader))
for images, targets in bar:
    images = list(image.to('cuda') for image in images)
    targets = [{k: v.to('cuda') for k, v in t.items()} for t in targets]

    with torch.no_grad():
        predictions = model(images)

    for i, prediction in enumerate(predictions):
        boxes = prediction['boxes'].cpu().numpy()
        masks = prediction['masks'].cpu().numpy()
        labels = prediction['labels'].cpu().numpy()

        # Process the predictions as needed
        print(f"Image {i}:")
        print("Boxes:", boxes)
        print("Masks:", masks)
        print("Labels:", labels)

    bar.update()

Inference:   0%|          | 0/53 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
model = torchvision.models.get_model(
        args.model, weights=args.weights, weights_backbone=args.weights_backbone, num_classes=num_classes, **kwargs
    )
model.roi_heads.box_predictor.cls_score = nn.Linear(in_features=1024, out_features=len(class_names),bias=True)
model.roi_heads.box_predictor.bbox_pred = nn.Linear(in_features=1024, out_features=len(class_names)*4,bias=True)
model.roi_heads.mask_predictor.mask_fcn_logits = nn.Conv2d(256, len(class_names),kernel_size=(1,1),stride=(1,1))

model.to(device)

In [ ]:
class EnhancedMaskRCNN(MaskRCNN):
    def __init__(self, backbone, num_classes=None, **kwargs):
        super().__init__(backbone, num_classes, **kwargs)
        # 添加邊界感知分支
        self.boundary_head = self._build_boundary_head()

    def _build_boundary_head(self):
        layers = [
            torch.nn.Conv2d(256, 256, 3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 256, 3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 1, 1)  # 邊界檢測輸出
        ]
        return torch.nn.Sequential(*layers)

    def forward(self, images, targets=None):
        # 原始輸出
        outputs = super().forward(images, targets)

        # 邊界檢測分支
        if self.training and targets is not None:
            boundary_maps = self.boundary_head(outputs['features'])
            outputs['boundary_loss'] = self.compute_boundary_loss(boundary_maps, targets)

        return outputs

def create_model(num_classes=5, pretrained=True):
    backbone = torchvision.models.resnet50(weights=ResNet50_Weights.DEFAULT)
    # backbone = torchvision.models._utils.IntermediateLayerGetter(
    #     backbone, return_layers={'layer4': 'out'}
    # )
    # backbone = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

    model = EnhancedMaskRCNN(
        backbone,
        num_classes=num_classes,
        min_size=512,
        max_size=512
    )

    # 修改分類頭
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # 修改mask頭
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask, 256, num_classes
    )

    return model


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = create_model().to(device)
params = [p for p in model.parameters() if p.requires_grad]

optimizer = SGD(params, lr=0.003, momentum=0.9, weight_decay=0.0005)
lr_sched = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# 混合損失函數
def hybrid_loss(pred_masks, gt_masks, boundaries):
    mask_loss = torch.nn.functional.binary_cross_entropy_with_logits(pred_masks, gt_masks)
    boundary_loss = torch.nn.functional.mse_loss(pred_masks, boundaries)
    return mask_loss + 0.3 * boundary_loss

NUM_EPOCHS = 30

for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0.0

    for images, targets in train_loader:
        images = list(img.to(device) for img in images)
        targets = [{'masks': t.to(device)} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        losses.backward()
        optimizer.step()

        total_loss += losses.item()

    lr_sched.step()
    print(f"Epoch {epoch+1} | Avg Loss: {total_loss/len(train_loader):.4f}")



AttributeError: 'ResNet' object has no attribute 'out_channels'

In [ ]:
def masks_to_coco(results, image_ids):
    coco_results = []
    for img_id, output in zip(image_ids, results):
        for score, mask, label in zip(output['scores'], output['masks'], output['labels']):
            rle = binary_mask_to_rle(mask)
            coco_results.append({
                "image_id": img_id,
                "category_id": label.item(),
                "segmentation": rle,
                "score": score.item()
            })
    return coco_results

def binary_mask_to_rle(mask):
    # RLE編碼實現
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return {'size': list(mask.shape[-2:]), 'counts': runs.tolist()}


In [ ]:
model.eval()
test_loader = DataLoader(test_set, batch_size=2, shuffle=False)

results = []
with torch.no_grad():
    for batch in test_loader:
        outputs = model(batch.to(device))
        results.extend(outputs)

# 生成最終提交文件
with open('test-results.json', 'w') as f:
    json.dump(masks_to_coco(results, test_set.image_ids), f)

print("Submission file generated!")